In [75]:
import os, re 
import numpy as np
import tensorflow as tf
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free']


In [76]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   
    if sentence[-1] == ":": continue 

    if idx > 9: break  
        
    print(sentence)

[Verse 1]
They come from everywhere
A longing to be free
They come to join us here
From sea to shining sea And they all have a dream
As people always will
To be safe and warm
In that shining city on the hill Some wanna slam the door
Instead of opening the gate
Aw, let's turn this thing around


In [77]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [78]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> verse <end>',
 '<start> they come from everywhere <end>',
 '<start> a longing to be free <end>',
 '<start> they come to join us here <end>',
 '<start> from sea to shining sea and they all have a dream <end>',
 '<start> as people always will <end>',
 '<start> to be safe and warm <end>',
 '<start> in that shining city on the hill some wanna slam the door <end>',
 '<start> instead of opening the gate <end>',
 '<start> aw , let s turn this thing around <end>']

In [79]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000, 
        filters=' ',
        oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  520    3 ...    0    0    0]
 [   2   45   66 ...    0    0    0]
 [   2    9 3390 ...    0    0    0]
 ...
 [   2  561   21 ...    0    0    0]
 [   2  120   34 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f51440d4a90>


In [80]:
print(tensor[:3, :10])
print(tensor[0])

[[   2  520    3    0    0    0    0    0    0    0]
 [   2   45   66   74  808    3    0    0    0    0]
 [   2    9 3390   10   27  270    3    0    0    0]]
[  2 520   3   0   0   0   0   0   0   0   0   0   0   0   0]


In [81]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [82]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle = True)

[  2 520   3   0   0   0   0   0   0   0   0   0   0   0]
[520   3   0   0   0   0   0   0   0   0   0   0   0   0]


In [83]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [84]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE


VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [85]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__() 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [86]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 20001), dtype=float32, numpy=
array([[[-1.29753244e-04, -1.06436906e-04,  1.65805250e-05, ...,
          1.13941678e-04, -3.55128868e-04, -4.70600244e-05],
        [-2.87334435e-04, -3.25193803e-04, -9.77944073e-05, ...,
          3.38731152e-05, -5.49725140e-04, -1.06889762e-04],
        [-2.57467997e-04, -3.27733869e-04, -1.25418635e-04, ...,
          2.53926177e-04, -7.64805358e-04, -4.05277169e-05],
        ...,
        [-2.54626560e-04,  6.00399158e-04,  1.04911486e-03, ...,
          1.25911145e-03,  1.72445012e-04,  8.14099854e-04],
        [-1.73412467e-04,  4.24438680e-04,  1.20585458e-03, ...,
          1.04389479e-03,  1.40085423e-04,  6.12118805e-04],
        [-9.41250164e-06,  3.55550961e-04,  1.21016288e-03, ...,
          9.55913216e-04, -1.50652137e-04,  2.91538803e-04]],

       [[-5.47748095e-05, -2.75493541e-04,  1.10274814e-04, ...,
          1.82984513e-05, -2.87319544e-05,  1.78663031e-04],
        [ 7.78241956e-05, -5.97832433e-04,  2

In [87]:
model.summary()

Model: "text_generator_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      multiple                  5120256   
_________________________________________________________________
lstm_12 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_13 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_6 (Dense)              multiple                  20501025  
Total params: 39,260,961
Trainable params: 39,260,961
Non-trainable params: 0
_________________________________________________________________


In [88]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 292s 423ms/step - loss: 4.2894
Epoch 2/10
686/686 [==============================] - 292s 426ms/step - loss: 3.2662
Epoch 3/10
686/686 [==============================] - 292s 425ms/step - loss: 3.0620
Epoch 4/10
686/686 [==============================] - 291s 424ms/step - loss: 2.9255
Epoch 5/10
686/686 [==============================] - 290s 423ms/step - loss: 2.7952
Epoch 6/10
686/686 [==============================] - 290s 423ms/step - loss: 2.6877
Epoch 7/10
686/686 [==============================] - 290s 422ms/step - loss: 2.5885
Epoch 8/10
686/686 [==============================] - 289s 421ms/step - loss: 2.4952
Epoch 9/10
686/686 [==============================] - 290s 423ms/step - loss: 2.4116
Epoch 10/10
686/686 [==============================] - 291s 424ms/step - loss: 2.3334


In [89]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [91]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '